In [ ]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
!tar -xvf images.tar


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory('Images',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory('Images',
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='categorical',
                                                         subset='validation')


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(120, activation='softmax'))


In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
model_checkpoint= ModelCheckpoint('dog_breed_classifier.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=100,
                    callbacks=[early_stopping, model_checkpoint])


In [ ]:

plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training Loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'r',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)



In [ ]:
# Load best saved model
model.load_weights('dog_breed_classifier.h5')

# Predict labels for validation dataset
preds = model.predict(validation_generator)

# Get classification report
print(classification_report(np.argmax(validation_generator.labels, axis=-1), np.argmax(preds, axis=-1)))


In [ ]:
from keras.models import load_model

model = load_model('dog_breed_classifier.h5')


In [ ]:
from keras.preprocessing import image

def load_and_process_image(image_path):
    # Load image
    img = image.load_img(image_path, target_size=(224, 224))
    # Convert image to numpy array
    img = image.img_to_array(img)
    # Add a dimension
    img = np.expand_dims(img, axis=0)
    # Preprocess image
    img = preprocess_input(img)
    return img


In [ ]:
# Load and process image
img = load_and_process_image('/path_to_your_image/image.jpg')

# Predict the class of the image
preds = model.predict(img)

# Get the index of the class with the highest probability
pred_class_index = np.argmax(preds[0])

# Get the label of the class from the generator's class_indices
pred_class = list(train_generator.class_indices.keys())[list(train_generator.class_indices.values()).index(pred_class_index)]

print("Predicted class:", pred_class)
